In [ ]:
import os, json
from pathlib import Path
from pandas import DataFrame
from mpcontribs.client import Client
from unflatten import unflatten

In [ ]:
client = Client()

**Load raw data**

In [ ]:
name = "screening_inorganic_pv"
indir = Path("/Users/patrick/gitrepos/mp/mpcontribs-data/ThinFilmPV")
files = {
    "summary": "SUMMARY.json",
    "absorption": "ABSORPTION-CLIPPED.json",
    "dos": "DOS.json",
    "formulae": "FORMATTED-FORMULAE.json"
}
data = {}

for k, v in files.items():
    path = indir / v
    with path.open(mode="r") as f:
        data[k] = json.load(f)
        
for k, v in data.items():
    print(k, len(v))

**Prepare contributions**

In [ ]:
config = {
    "SLME_500_nm": {"path": "SLME.500nm", "unit": "%"},
    "SLME_1000_nm": {"path": "SLME.1000nm", "unit": "%"},
    "E_g": {"path": "ΔE.corrected", "unit": "eV"},
    "E_g_d": {"path": "ΔE.direct", "unit": "eV"},
    "E_g_da": {"path": "ΔE.dipole", "unit": "eV"},
    "m_e": {"path": "mᵉ", "unit": "mₑ"},
    "m_h": {"path": "mʰ", "unit": "mₑ"}
}
columns = {c["path"]: c["unit"] for c in config.values()}
contributions = []

for mp_id, d in data["summary"].items():
    formula = data["formulae"][mp_id].replace("<sub>", "").replace("</sub>", "")
    contrib = {"project": name, "identifier": mp_id, "data": {"formula": formula}}
    cdata = {v["path"]: f'{d[k]} {v["unit"]}' for k, v in config.items()}
    contrib["data"] = unflatten(cdata)
        
    df_abs = DataFrame(data=data["absorption"][mp_id])
    df_abs.columns = ["hν [eV]", "α [cm⁻¹]"]
    df_abs.set_index("hν [eV]", inplace=True)
    df_abs.columns.name = "" # legend name
    df_abs.attrs["name"] = "absorption"
    df_abs.attrs["title"] = "optical absorption spectrum"
    df_abs.attrs["labels"] = {"variable": "", "value": "α [cm⁻¹]"}

    df_dos = DataFrame(data=data["dos"][mp_id])
    df_dos.columns = ['E [eV]', 'DOS [eV⁻¹]']
    df_dos.set_index("E [eV]", inplace=True)
    df_dos.columns.name = "" # legend name
    df_dos.attrs["name"] = "DOS"
    df_dos.attrs["title"] = "electronic density of states"
    df_dos.attrs["labels"] = {"variable": "", "value": "DOS [eV⁻¹]"}

    contrib["tables"] = [df_abs, df_dos]
    contributions.append(contrib)
    
len(contributions)

**Submit contributions**

In [ ]:
client.delete_contributions(name)
client.init_columns(name, columns)
client.submit_contributions(contributions[:5])

**Retrieve and plot tables**

In [ ]:
all_ids = client.get_all_ids(
    {"project": "screening_inorganic_pv"}, include=["tables"]
).get(name, {})
cids = list(all_ids["ids"])
tids = list(all_ids["tables"]["ids"])
len(cids), len(tids)

In [ ]:
client.get_contribution(cids[0])

In [ ]:
t = client.get_table(tids[0])  # pandas DataFrame

In [ ]:
t.display()